In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("Batch47_CSE7322c_CUTe").setMaster('local[*]').set("spark.driver.extraClassPath","/home/datasets/mysql-connector-java-8.0.13.jar").set("spark.executor.extraClassPath","/home/datasets/mysql-connector-java-8.0.13.jar")
sc = SparkContext(conf=conf)  
spark = SparkSession(sc)  

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd

In [4]:
item=  spark.read.format("jdbc").options(url="jdbc:mysql://172.16.0.218:3306/insofe_web_sales_db",driver="com.mysql.jdbc.Driver",dbtable="item",user="insofeadmin",password="insofe_password").load()

In [5]:
warehouse=  spark.read.format("jdbc").options(url="jdbc:mysql://172.16.0.218:3306/insofe_web_sales_db",driver="com.mysql.jdbc.Driver",dbtable="warehouse",user="insofeadmin",password="insofe_password").load()

In [6]:
web_sales=  spark.read.format("jdbc").options(url="jdbc:mysql://172.16.0.218:3306/insofe_web_sales_db",driver="com.mysql.jdbc.Driver",dbtable="web_sales",user="insofeadmin",password="insofe_password").load()

In [22]:
web_sales.createOrReplaceTempView("web_sales")

In [18]:
warehouse.createOrReplaceTempView("warehouseTable")

### 1. Find the top 10 items (ws_item_sk) that got sold online and their count from the web_sales table.

In [7]:
web_sales.createOrReplaceTempView("web_salesTable")

In [8]:
spark.sql("""select ws_item_sk,count(*) as count from web_salesTable group by ws_item_sk order by count desc limit 10""").show()

+----------+-----+
|ws_item_sk|count|
+----------+-----+
|     16339|  115|
|     10657|  112|
|     15091|  111|
|      4861|  111|
|      3091|  110|
|      2149|  109|
|     16081|  108|
|     14683|  108|
|     15361|  108|
|     15655|  107|
+----------+-----+



### 2. Group the items based on brand(i_brand) and display the top 50 brands. (Use item table) 

In [9]:
item.createOrReplaceTempView("itemTable")

In [10]:
spark.sql("""select i_brand from itemTable group by i_brand order by i_brand desc limit 50""").show()

+-----------------+
|          i_brand|
+-----------------+
| univunivamalg #9|
| univunivamalg #8|
| univunivamalg #7|
| univunivamalg #6|
| univunivamalg #5|
| univunivamalg #4|
| univunivamalg #3|
| univunivamalg #2|
|univunivamalg #12|
|univunivamalg #11|
|univunivamalg #10|
| univunivamalg #1|
|  univnameless #9|
|  univnameless #8|
|  univnameless #7|
|  univnameless #6|
|  univnameless #5|
|  univnameless #4|
|  univnameless #3|
|  univnameless #2|
+-----------------+
only showing top 20 rows



### 3. Display the top 50 items in each brand and class.

In [11]:
spark.sql("select i_brand, i_class, count(*) as count from itemTable group by i_brand, i_class order by count desc limit 50").show()

+-------------------+--------------+-----+
|            i_brand|       i_class|count|
+-------------------+--------------+-----+
|  importoscholar #2|       country|  253|
| edu packscholar #1|     classical|  253|
|  importoimporto #2|        shirts|  250|
|  exportischolar #1|           pop|  245|
| importoedu pack #2|          mens|  243|
| edu packimporto #1|sports-apparel|  242|
|  importoimporto #1|        shirts|  242|
| edu packscholar #2|     classical|  242|
|  exportiexporti #1|      toddlers|  241|
| importoedu pack #1|          mens|  239|
|    importoamalg #2|    fragrances|  238|
|  exportischolar #2|           pop|  238|
|   amalgedu pack #2|        womens|  234|
|   edu packamalg #2|      swimwear|  233|
|      amalgamalg #1|       dresses|  227|
|    importoamalg #1|    fragrances|  227|
|edu packedu pack #2|      athletic|  227|
|   edu packamalg #1|      swimwear|  227|
|    amalgexporti #1|       newborn|  225|
|    amalgscholar #1|          rock|  225|
+----------

### 4. Display top 20 items that got sold online ordered based on Total Quantity 

In [12]:
spark.sql("""select ws_item_sk, sum(ws_quantity) as total_quantity from web_salesTable group by ws_item_sk order by total_quantity desc limit 20""").show()

+----------+--------------+
|ws_item_sk|total_quantity|
+----------+--------------+
|      3091|          6007|
|      6907|          5876|
|      2677|          5831|
|      4861|          5818|
|      5485|          5778|
|     14683|          5755|
|      2965|          5680|
|     12583|          5630|
|      6523|          5619|
|       625|          5600|
|     12265|          5580|
|      3739|          5576|
|      6535|          5572|
|      2305|          5545|
|     16339|          5524|
|     14047|          5505|
|     17479|          5503|
|     13729|          5498|
|      5515|          5491|
|      5929|          5471|
+----------+--------------+



### 5. Display the count of classes for each category of items.

In [13]:
spark.sql(""" select count(i_class), i_category from itemTable group by i_category""").show()

+--------------+-----------+
|count(i_class)| i_category|
+--------------+-----------+
|          1804|       Home|
|          1781|     Sports|
|            22|       null|
|          1812|Electronics|
|          1731|      Books|
|          1809|        Men|
|          1856|      Music|
|          1789|      Women|
|          1833|      Shoes|
|          1736|    Jewelry|
|          1784|   Children|
+--------------+-----------+



### 6. Display the i_brand, i_wholesale_cost, i_class for the top 25 items that got sold online along with their count. (Use web_sales and item taes)

In [14]:
spark.sql("select i.i_brand,i.i_class,ws.ws_wholesale_cost,count(*) as count from itemTable i join web_salesTable ws on i.i_item_sk = ws.ws_item_sk group by i.i_brand,i.i_class,ws.ws_wholesale_cost order by count desc").show(25)

+-------------------+---------------+-----------------+-----+
|            i_brand|        i_class|ws_wholesale_cost|count|
+-------------------+---------------+-----------------+-----+
|   amalgedu pack #2|         womens|            62.37|   10|
| importoedu pack #2|           mens|            16.55|    8|
|    amalgexporti #2|        newborn|            91.28|    8|
| importoedu pack #2|           mens|            49.88|    8|
|  exportiimporto #2|          pants|            87.95|    7|
| edu packimporto #2| sports-apparel|            38.97|    7|
|  importoimporto #1|         shirts|            96.45|    7|
|  importoimporto #2|         shirts|             9.47|    7|
| edu packscholar #2|      classical|            90.20|    7|
| edu packscholar #2|      classical|            20.63|    7|
|  importoimporto #2|         shirts|            30.56|    7|
|  importoscholar #2|        country|            58.75|    7|
|      amalgamalg #2|        dresses|            83.34|    7|
| export

### 7. Display the top 25 brands (i_brand) that got sold online along with their count. 

In [27]:
spark.sql("select i.i_brand,count(*) as count from itemTable i join web_sales ws on i.i_item_sk = ws.ws_item_sk group by i.i_brand order by count desc").show(25)

+-------------------+-----+
|            i_brand|count|
+-------------------+-----+
| importoedu pack #2|12172|
| edu packscholar #2|11946|
|  exportischolar #2|11725|
|  importoscholar #2|11702|
|  importoimporto #2|11567|
|    importoamalg #2|11477|
|   edu packamalg #2|11287|
|   amalgedu pack #2|10996|
|  importoexporti #2|10885|
|      amalgamalg #2|10878|
| edu packexporti #2|10873|
|edu packedu pack #2|10731|
| exportiedu pack #2|10641|
|  exportiimporto #2|10490|
|    amalgexporti #2|10400|
|    amalgimporto #2|10131|
|  exportiexporti #2| 9902|
| edu packimporto #2| 9873|
|    exportiamalg #2| 9795|
|    amalgscholar #2| 9724|
| edu packscholar #1| 8215|
| edu packimporto #1| 8036|
|  exportiexporti #1| 8036|
|  importoimporto #1| 7904|
|  exportischolar #1| 7813|
+-------------------+-----+
only showing top 25 rows



### 8. Display the top 25 classes (i_class) that got sold online along with their count.

In [16]:
spark.sql("select i_class,count(*) as count from itemTable group by i_class order by count desc limit 25").show()

+---------------+-----+
|        i_class|count|
+---------------+-----+
|           kids|  538|
|      classical|  495|
|         shirts|  492|
|           mens|  483|
|            pop|  483|
|     fragrances|  465|
|       swimwear|  460|
|         womens|  459|
|       toddlers|  459|
|        country|  451|
|       athletic|  451|
|        dresses|  449|
|        newborn|  448|
| sports-apparel|  444|
|    accessories|  443|
|school-uniforms|  440|
|        infants|  437|
|          pants|  432|
|           rock|  428|
|      maternity|  418|
+---------------+-----+
only showing top 20 rows



### 9. Display the sum of the ws_ext_sales_price for each class(i_class) that got sold online. 

In [17]:
spark.sql("select i.i_class,sum(ws.ws_ext_sales_price) as count from itemTable as i join web_salesTable as ws on i.i_item_sk = ws.ws_item_sk group by i.i_class") .show()

+---------------+-----------+
|        i_class|      count|
+---------------+-----------+
|          pools|12911573.77|
|         travel|11518267.35|
|         hockey|10586186.81|
|     mattresses|11773261.56|
|school-uniforms|45462865.77|
|    televisions|11569919.80|
|        newborn|43965615.69|
|     automotive| 9807940.43|
|    consignment| 9423738.24|
|       athletic|44936971.82|
|         estate|10703558.23|
|            pop|49915680.94|
|  jewelry boxes|13040282.69|
|        camping|11071930.19|
|      computers|13332186.90|
|dvd/vcr players|14174158.21|
|         stereo|12832302.53|
|     fragrances|47685567.74|
|       scanners|10719848.69|
|      bracelets|10163329.84|
+---------------+-----------+
only showing top 20 rows



### 10. Display the classes (i_class) and the net profit for the items sold online and their warehouse information (ws_warehouse_sk).

In [24]:
spark.sql("select i.i_item_sk,i.i_class,sum(ws.ws_net_profit) as sum,w.w_warehouse_sk from itemTable as i join web_sales as ws on i.i_item_sk = ws.ws_item_sk join warehouseTable as w on ws.ws_warehouse_sk = w.w_warehouse_sk group by i.i_item_sk,i.i_class,w.w_warehouse_sk order by i.i_item_sk,sum desc").show(50)


+---------+--------+---------+--------------+
|i_item_sk| i_class|      sum|w_warehouse_sk|
+---------+--------+---------+--------------+
|        1|     pop|  -619.56|             4|
|        1|     pop|  -725.30|             5|
|        1|     pop| -2022.64|             1|
|        1|     pop| -6205.65|             3|
|        1|     pop| -9058.65|             2|
|        2| dresses|  1567.22|             1|
|        2| dresses|   721.88|             5|
|        2| dresses| -3453.57|             3|
|        2| dresses| -4212.21|             4|
|        2| dresses|-11504.05|             2|
|        3|   decor|  2492.37|             1|
|        3|   decor|  -266.68|             3|
|        3|   decor|  -415.32|             2|
|        3|   decor|-15248.34|             5|
|        3|   decor|-16343.92|             4|
|        4| infants|  2824.68|             3|
|        4| infants|    63.87|             2|
|        4| infants| -1158.48|             5|
|        4| infants| -1173.90|    

In [ ]:
# sqoop job \
# --create cute5_importalltables_2305_2 \
#         -- import-all-tables \
#         --connect jdbc:mysql://172.16.0.218:3306/insofe_web_sales_db \
#         --username insofeadmin \
#         -P \
#         --warehouse-dir '/user/2305B47/insofe_web_sales_db/'
# sqoop job --exec cute5_importalltables_2305_2            

In [ ]:
# jupyter notebook --no-browser --ip 172.16.0.218 --port 22305